In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from utils import group_into_trips, group_by_engine_type
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
static = pd.read_csv('../VED_Static_Data_ICE_HEV.csv')
timeseries = pd.read_csv('../sample_data.csv')
ICEs, _, _, _ = group_by_engine_type(static, timeseries)

In [14]:
trips = group_into_trips(ICEs)

In [15]:
print(trips[0]['Timestamp(ms)'].head(100))
print(trips[1]['Timestamp(ms)'].head())

0         0
1       200
2      2200
3      2800
4      3000
      ...  
95    69700
96    70000
97    70700
98    71800
99    72800
Name: Timestamp(ms), Length: 100, dtype: int64
69763       0
69764     800
69765    1200
69766    1800
69767    1900
Name: Timestamp(ms), dtype: int64


In [16]:
def accel_functor():
    '''
    Return a function object that, when called with a vehicle's velocity (in km/h) 
    and a timestamp (in ms), computes the acceleration of the vehicle (in mph/s)
    based on the velocity and time of the given parameters and the parameters from
    the previous call to the function.

    The reason for the units of mph/s is for compatibility with the power factor
    formula as presented in the following paper:
    https://silo.tips/download/a-comparison-of-real-world-and-modeled-emissions-under-conditions-of-variable-dr


    This is very convenient to use with Pandas Series objects.
    For example (where trip is a DataFrame representing a trip):
    f = accel_functor()
    a = trip.apply(lambda df : f(df['Vehicle Speed[km/h]'], df['Timestamp(ms)']), axis=1)
    '''
    unit_conversion = 621.371 # from kph / ms to mph / s

    prev_v = None
    prev_t = None
    def wrapped(v, t):
        nonlocal prev_v, prev_t
        a = None
        if prev_v is not None and prev_t is not None:
            delta_v = v - prev_v # in km/h
            delta_t = t - prev_t # in ms

            a = delta_v / delta_t * unit_conversion
        prev_v = v
        prev_t = t
        return a
    return wrapped

def get_accel(trip):
    '''
    Given a trip as a Pandas DataFrame, return a Pandas Series that contains acceleration 
    data for the vehicle (in mph/s) on that trip.

    The reason for the units of mph/s is for compatibility with the power factor
    formula as presented in the following paper:
    https://silo.tips/download/a-comparison-of-real-world-and-modeled-emissions-under-conditions-of-variable-dr
    '''
    trip.sort_values(by=['Timestamp(ms)']) # this might be redundant (better safe than sorry)
    f = accel_functor()
    return trip.apply(lambda df : f(df['Vehicle Speed[km/h]'], df['Timestamp(ms)']), axis=1)

def power_factor(trip):
    '''
    Given a trip as a Pandas DataFrame and acceleration as a Pandas Series, return power
    factor data for the trip as a Pandas Series.
    '''
    trip.sort_values(by=['Timestamp(ms)']) # this might be redundant (better safe than sorry)
    return trip.apply(lambda df : 2 * df['Vehicle Speed[km/h]'] * df['Acceleration[mph/s]'], axis=1)

def aggressivity(power_factors):
    '''
    Given a timeseries of power factors as a pandas Series, return the aggressivity of the trip
    '''
    return np.sqrt(np.sum(power_factors ** 2) / len(power_factors)) 


In [17]:
trip = trips[0]
accel = get_accel(trip)
trip['Acceleration[mph/s]'] = accel
pf = power_factor(trip)
print(pf.head())
print(aggressivity(pf))

0          NaN
1   -571.66132
2     31.06855
3      0.00000
4      0.00000
dtype: float64
311.8196681425237


In [18]:
aggressivities = []
for trip in trips:
    accel = get_accel(trip)
    trip['Acceleration[mph/s]'] = accel
    pf = power_factor(trip)
    aggressivity_score = aggressivity(pf)
    aggressivities.append(aggressivity_score)
    veh_id = trip['VehId'].unique()[0]
    trip_id = trip['Trip'].unique()[0]

    print(f"VehId: {veh_id}, TripId: {trip_id} aggressivity: {aggressivity_score}")

TypeError: 'builtin_function_or_method' object is not subscriptable

In [2]:
plt.plot(aggressivity_score[:20])

NameError: name 'aggressivity' is not defined